# Seras学院 データ解析 (Data Analysis)

共通モジュール `loader.py` を使用してデータを読み込みます。
自動的に型変換（Timestamp -> Datetime, Date -> Date）が行われます。

In [ ]:
import polars as pl
from loader import load_data

# データ読み込み
# .envの読み込みや認証は loader.py 側で行われます
df_occupancy, df_open = load_data()

print("Data loaded successfully.")

In [ ]:
print("--- Occupancy Logs ---")
display(df_occupancy)
print(df_occupancy.schema)

print("\n--- Open Logs ---")
display(df_open)
print(df_open.schema)